In [1]:
from src import geneticAlgo, follow_time
from src import plot_outcomes
import matplotlib.pyplot as plt
import tqdm
import pickle
import numpy as np
import pandas as pd

### Genetic algorithm

In [5]:
size_population = 200000
ratio_breeding = 0.7
size_tournament = 20000
gene_size = 30

algo = geneticAlgo(size_population = size_population, 
                   ratio_breeding = ratio_breeding, 
                   size_tournament = size_tournament,
                   gene_size = gene_size)

Outcome test symmetry: Validated.


In [6]:
D = {}
for option in algo.L_options:
    print(option.capitalize())
    D[option] = []
    for _ in tqdm.tqdm(range(10)):
        dt = []
        f0 = follow_time()
        algo.create_population()
        dt.append(f0.update())
        for i in range(20):
            algo.next_gen(option = option)
            dt.append(f0.update())
        ind = algo.indicators
        ind["time"] = dt

        D[option].append(ind)

Uniform


100%|██████████| 10/10 [01:16<00:00,  7.66s/it]


Elitist


100%|██████████| 10/10 [01:14<00:00,  7.46s/it]


Lottery


100%|██████████| 10/10 [01:31<00:00,  9.16s/it]


Tournament


100%|██████████| 10/10 [03:50<00:00, 23.10s/it]


Random


100%|██████████| 10/10 [01:47<00:00, 10.78s/it]


In [9]:
with open('Analysis/output.pickle', 'wb') as handle:
    pickle.dump(D, handle, protocol=pickle.HIGHEST_PROTOCOL)

### NEXT STEPS
- Implement TSP2
- Improve computing power using cuda (voir)

### Analysis

In [12]:
with open('Analysis/output.pickle', 'rb') as f:
    D = pickle.load(f)

In [3]:
# Figures indicators
for name_indicator in ['best_fitness', 'average_fitness', 'fitness_variance']:
    fig = plot_outcomes(D, name_indicator)
    plt.savefig(f"Analysis/{name_indicator}.png", dpi = 300); plt.close()

In [18]:
D_t = {}
for n, option in enumerate(D.keys()):
    out = np.zeros((21,10))

    for i, k in enumerate(D[option]):
        data = np.array(k["time"])
        out[:,i] = data
    D_opt = {"initialization (s.)":np.mean(out[0,:]), 
             "mean new gen. (s.)":np.mean(out[1:,:]), 
             "min new gen. (s.)":np.min(out[1:,:]), 
             "max new gen. (s.)":np.max(out[1:,:])}    
    D_t[option] = D_opt

In [20]:
df_time = pd.DataFrame.from_dict(D_t)
display(df_time)

,uniform,elitist,lottery,tournament,random
initialization (s.),0.540111,0.529841,0.527267,0.563535,0.516741
mean new gen. (s.),0.355836,0.346682,0.431739,1.126733,0.513094
min new gen. (s.),0.305813,0.331343,0.416751,1.024163,0.493253
max new gen. (s.),3.943145,1.563854,0.528138,2.898793,0.611241
